# BentoML 통합 가이드

[BentoML](https://github.com/bentoml/BentoML/)은 AI 애플리케이션을 구축, 배포 및 확장하기 위해 설계된 오픈 소스 프레임워크입니다. 사용자는 확산 모델을 프로덕션용으로 쉽게 패키징하고 제공하여 안정적이고 효율적인 배포를 보장할 수 있습니다. 모니터링 및 추적과 같은 즉시 사용 가능한 운영 관리 도구를 제공하며 다양한 클라우드 플랫폼에 쉽게 배포할 수 있도록 지원합니다. BentoML의 분산 아키텍처와 API 서버 로직과 모델 추론 로직의 분리는 예산 제약 조건에서도 배포를 효율적으로 확장할 수 있도록 합니다. 결과적으로 Diffusers와 통합하면 실제 배포에 유용한 도구를 제공합니다.

이 튜토리얼에서는 BentoML을 Diffusers와 통합하는 방법을 보여줍니다.

## 필수 조건

- [Diffusers](https://huggingface.co/docs/diffusers/installation) 설치합니다.
- `pip install bentoml`을 실행하여 BentoML을 설치합니다. 자세한 내용은 [BentoML 설명서](https://docs.bentoml.com)를 참조하세요.

## 확산 모델 가져오기

먼저 모델을 준비해야 합니다. BentoML에는 모델 관리를 위한 자체 [모델 저장소](https://docs.bentoml.com/en/latest/concepts/model.html)가 있습니다. 아래와 같이 `download_model.py` 파일을 만들어 확산 모델을 BentoML의 모델 저장소로 가져옵니다.

In [ ]:
import bentoml

bentoml.diffusers.import_model(
    "sd2.1",  # BentoML 모델 저장소의 모델 태그
    "stabilityai/stable-diffusion-2-1",  # Hugging Face 모델 식별자
)

이 코드 조각은 Hugging Face Hub에서 Stable Diffusion 2.1 모델(리포지토리 ID `stabilityai/stable-diffusion-2-1` 사용)을 다운로드하거나(모델이 이미 다운로드된 경우 캐시된 다운로드 파일 사용) BentoML 모델 저장소에 `sd2.1`이라는 이름으로 가져옵니다.

이미 미세 조정되어 디스크에 저장된 모델의 경우 리포지토리 ID 대신 경로를 제공할 수 있습니다.

In [ ]:
import bentoml

bentoml.diffusers.import_model(
    "sd2.1-local",
    "./local_stable_diffusion_2.1/",
)

모델 저장소에서 모델을 볼 수 있습니다.

```
bentoml models list

Tag                                                                 Module                              Size       Creation Time       
sd2.1:ysrlmubascajwnry                                              bentoml.diffusers                   33.85 GiB  2023-07-12 16:47:44 
```

## BentoML을 사용하여 확산 모델을 RESTful 서비스로 전환

확산 모델이 BentoML의 모델 저장소에 있으면 이를 사용하여 텍스트-이미지 서비스를 구현할 수 있습니다. Stable Diffusion 모델은 이미지 생성 프로세스를 안내하기 위해 필요한 프롬프트 외에도 다양한 인수를 허용합니다. 이러한 입력 인수를 유효성 검사하려면 BentoML의 [pydantic](https://github.com/pydantic/pydantic) 통합을 사용합니다. 예제 pydantic 모델로 `sdargs.py` 파일을 만듭니다.

In [ ]:
import typing as t

from pydantic import BaseModel


class SDArgs(BaseModel):
    prompt: str
    negative_prompt: t.Optional[str] = None
    height: t.Optional[int] = 512
    width: t.Optional[int] = 512

    class Config:
        extra = "allow"

이 pydantic 모델은 문자열 필드 `prompt`와 세 가지 선택적 필드 `height`, `width`, `negative_prompt`가 필요하며 각 필드에는 해당 유형이 있습니다. `extra = "allow"` 줄은 `SDArgs` 클래스에 정의되지 않은 추가 필드를 추가하는 것을 지원합니다. 실제 시나리오에서는 원하는 모든 필드를 정의하고 추가 필드를 허용하지 않을 수 있습니다.

다음으로 Stable Diffusion 서비스를 정의하는 BentoML 서비스 파일을 만듭니다.

In [ ]:
import bentoml
from bentoml.io import Image, JSON

from sdargs import SDArgs

bento_model = bentoml.diffusers.get("sd2.1:latest")
sd21_runner = bento_model.to_runner(name="sd21-runner")

svc = bentoml.Service("stable-diffusion-21", runners=[sd21_runner])


@svc.api(input=JSON(pydantic_model=SDArgs), output=Image())
async def txt2img(input_data):
    kwargs = input_data.dict()
    res = await sd21_runner.async_run(**kwargs)
    images = res[0]
    return images[0]

파일을 `service.py`로 저장하고 다음을 사용하여 BentoML 서비스 엔드포인트를 시작합니다.

```
bentoml serve service:svc
```

JSON 사전을 허용하는 `/txt2img` 엔드포인트가 있는 HTTP 서버가 포트 3000에서 실행되어야 합니다. 웹 브라우저에서 <http://127.0.0.1:3000>으로 이동하여 Swagger UI에 액세스합니다.

`curl`을 사용하여 텍스트-이미지 생성을 테스트하고 반환된 이미지를 `output.jpg`에 쓸 수도 있습니다.

```
curl -X POST http://127.0.0.1:3000/txt2img \
     -H 'Content-Type: application/json' \
     -d "{\"prompt\":\"a black cat\", \"height\":768, \"width\":768}" \
     --output output.jpg
```

## 클라우드 배포를 위한 BentoML 서비스 패키징

BentoML 서비스를 배포하려면 모든 소스 코드, 모델, 데이터 파일 및 종속성이 포함된 파일 아카이브인 BentoML [Bento](https://docs.bentoml.com/en/latest/concepts/bento.html)로 패키징해야 합니다. 이는 다음과 같이 `bentofile.yaml` 파일을 제공하여 수행할 수 있습니다.

```yaml
service: "service.py:svc"
include:
  - "service.py"
python:
  packages:
    - torch
    - transformers
    - accelerate
    - diffusers
    - triton
    - xformers
    - pydantic
docker:
    distro: debian
    cuda_version: "11.6"
```

`bentofile.yaml` 파일에는 패키지 종속성 및 Docker 옵션과 같은 [Bento 빌드 옵션](https://docs.bentoml.com/en/latest/concepts/bento.html#bento-build-options)이 포함되어 있습니다.

그런 다음 다음을 사용하여 Bento를 빌드합니다.

```
bentoml build
```

출력은 다음과 같습니다.

```
Successfully built Bento(tag="stable-diffusion-21:crkuh7a7rw5bcasc").

Possible next steps:

 * Containerize your Bento with `bentoml containerize`:
    $ bentoml containerize stable-diffusion-21:crkuh7a7rw5bcasc

 * Push to BentoCloud with `bentoml push`:
    $ bentoml push stable-diffusion-21:crkuh7a7rw5bcasc
```

다음 명령을 실행하여 Bento를 기반으로 Docker 이미지를 만들고 클라우드 공급자에 배포할 수 있습니다.

```
bentoml containerize stable-diffusion-21:crkuh7a7rw5bcasc
```

모델 배포 및 관리를 위한 엔드투엔드 솔루션을 원한다면 Bento를 [Yatai](https://github.com/bentoml/Yatai) 또는 [BentoCloud](https://bentoml.com/cloud)에 푸시하여 분산 배포할 수 있습니다.

BentoML과 Diffusers의 통합에 대한 자세한 내용은 [BentoML Diffusers 가이드](https://docs.bentoml.com/en/latest/frameworks/diffusers.html)를 참조하세요.